In [ ]:
!sudo pip install h5py

In [ ]:
import os
import sys
sys.path.append('../code/')

from nn_models import *
from nn_train_utils import *
from kinematics_decoding import load_model

from collections import defaultdict
import glob
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

**Make sure that the validation/training accuracy and test accuracy are very similar.**

In [ ]:
conv_folders = glob.glob('./experiment_1/*_*_*')

conv_keylist = ['arch_type', 'nlayers', 'n_skernels', 'n_tkernels', 
                's_kernelsize', 't_kernelsize', 's_stride', 't_stride', 
                'validation_accuracy']

In [ ]:
conv_dict = defaultdict(list)

for folder in conv_folders:
    try:
        with open(os.path.join(folder, 'config.yaml')) as myfile:
            myconfig = yaml.load(myfile)    
    except:
        print(folder)    
    for key in conv_keylist:
        conv_dict[key].append(myconfig[key])

In [ ]:
all_conv_models = pd.DataFrame(conv_dict)

all_conv_models = all_conv_models.astype({'nlayers': int, 
                                          's_kernelsize':int, 
                                          't_kernelsize': int, 
                                          's_stride': int, 
                                          't_stride': int})

In [ ]:
# Load train dataset as test
PATH_TO_DATA = os.path.abspath('../dataset/pcr_dataset_train.hdf5')
train_dataset = Dataset(os.path.join(PATH_TO_DATA, 'pcr_dataset_train.hdf5'), dataset_type='test')

conv_train_accuracies = [evaluate_model(
    load_model(all_conv_models.iloc[i], 1, 'conv'), train_dataset) for i in range(all_conv_models.shape[0])]

In [ ]:
# Load test dataset
PATH_TO_DATA = os.path.abspath('../dataset/pcr_dataset_test.hdf5')
test_dataset = Dataset(os.path.join(PATH_TO_DATA, 'pcr_dataset_test.hdf5'), dataset_type='test')

conv_test_accuracies = [evaluate_model(
    load_model(all_conv_models.iloc[i], 1, 'conv'), test_dataset) for i in range(all_conv_models.shape[0])]

In [ ]:
conv_val_accuracies = all_conv_models['validation_accuracy'].tolist()
all_conv_models['test_accuracy'] = pd.Series(conv_test_accuracies)
all_conv_models['train_accuracy'] = pd.Series(conv_train_accuracies)

In [ ]:
def num_parameters(model_path):
    path_to_ckptfile = os.path.join(model_path, 'model.ckpt')
    ckptReader = pywrap_tensorflow.NewCheckpointReader(path_to_ckptfile)
    shape_map = ckptReader.get_variable_to_shape_map()
    all_keys = list(shape_map.keys())
    weight_keys = [key for key in all_keys if 'weights' in key]
            
    useful_keys = [key for key in weight_keys if 'Adam' not in key]
    useful_keys = [key for key in useful_keys if 'Classifier' not in key]
    num_parameters = 0
    for key in useful_keys:
        num_parameters += np.product(shape_map[key])
        
    if num_parameters == np.nan:
        print('Number of parameters could not be computed!!')
        
    return num_parameters

num_pars_conv = []
for i in range(models.shape[0]):
    mymodel = load_model(all_conv_models.iloc[i], 1, 'conv', True)
    num_pars_conv.append(num_parameters(mymodel.model_path))

all_conv_models['parameters'] = pd.Series(num_pars_conv, index=all_conv_models.index, dtype=int)

In [ ]:
all_conv_models.to_pickle('convmodels.p')